In [1]:
import findspark
findspark.init()
findspark.find()

from pyspark import SparkContext

from pyspark.sql import SparkSession

from pyspark import SparkConf

import pyspark.sql as pysql

from pyspark import StorageLevel

In [2]:
import os
import sys

#os.environ['PYSPARK_PYTHON'] = sys.executable
#os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
conf = SparkConf().setAppName("Tarea") \
    .setMaster("local[8]") \
    .set("spark.executor.cores","2") \
    .set("spark.executor.memory", "4g") \
    .set("spark.driver.memory","4g")

In [4]:
spark : SparkSession = SparkSession.builder.config(conf=conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/21 00:06:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

In [6]:
columnas = [
    'Airline', # Aerolinea
    'CRSDepTime', #Hora de salida programada
    'DepTime', #Hora de salida actual
    'DepDelay', #Diferencia en minutos entre CRSDepTime y DepTime, tiempos negativos significa salidas tempranas
    'CRSArrTime',
    'ArrTime', #Hora de llegada
    'ArrDelay', #Diferencia de minutos entre la llegada programada y la hora de llegada, tiempos negativos significa llegadas tempranas
    'ActualElapsedTime', #Tiempo de vuelo verdadero
    'CRSElapsedTime', #Tiempo de vuelo programado
    'Distance', #Distancia entre aeropuertos en millas
    'Year', #Año del vuelo
    'Month', #Mes del vuelo
    'DayofMonth', #Dia del mes del vuelo
    'DayOfWeek', #Dia de la semana del vuelo
    'Tail_Number', #Numero de la cola, código para identificar la aeronave
    'Flight_Number_Operating_Airline', #Numero del vuelo
    
    'Origin', #Aeropuerto de origen
    'OriginAirportID', 'OriginAirportSeqID', #codigos del aeropuerto de origen
    'OriginCityName', #Nombre de la ciudad del aeropuerto de origen
    'OriginStateName', #Nombre de estado del aeropuerto de origen
    
    'Dest', #Aeropuerto de destino
    'DestAirportID', 'DestAirportSeqID', #codigos del aeropuerto de destino
    'DestCityName', #Nombre de la ciudad del aeropuerto de destino
    'DestStateName', #Nombre de estado del aeropuerto de destino
    
    'Cancelled', #El vuelo fue cancelado, 1 = Sí
    
    'Diverted', #Si el vuelo fue desviado, 1 = Sí
]

In [7]:
import zipfile
import pathlib

direccion = pathlib.Path.cwd()

path_datos_zip = pathlib.Path.joinpath(direccion.parent, 'Datos', 'data.zip')

path_guardar = pathlib.Path.joinpath(direccion.parent, 'Datos')

archivos_datos = [
    pathlib.Path.joinpath(direccion.parent, 'Datos', 'Combined_Flights_2022.csv').as_posix(),
    pathlib.Path.joinpath(direccion.parent, 'Datos', 'Combined_Flights_2021.csv').as_posix(),
    pathlib.Path.joinpath(direccion.parent, 'Datos', 'Combined_Flights_2020.csv').as_posix(),
    pathlib.Path.joinpath(direccion.parent, 'Datos', 'Combined_Flights_2019.csv').as_posix(),
    pathlib.Path.joinpath(direccion.parent, 'Datos', 'Combined_Flights_2018.csv').as_posix()
]

archivos_datos_en_zip  = [
    'Combined_Flights_2022.csv',
    'Combined_Flights_2021.csv',
    'Combined_Flights_2020.csv',
    'Combined_Flights_2019.csv',
    'Combined_Flights_2018.csv'
]

In [8]:
with zipfile.ZipFile(path_datos_zip, mode="r") as archivo_zip:
    for archivo in archivos_datos_en_zip:
        
        if os.path.isfile(path=pathlib.Path.joinpath(path_guardar,archivo).as_posix()):
            continue
        
        archivo_zip.extract(archivo, path_guardar)

In [9]:
archivos_datos

['/home/Andree/Documents/Maestria/DatosMasivos/DatosMasivos/Datos/Combined_Flights_2022.csv',
 '/home/Andree/Documents/Maestria/DatosMasivos/DatosMasivos/Datos/Combined_Flights_2021.csv',
 '/home/Andree/Documents/Maestria/DatosMasivos/DatosMasivos/Datos/Combined_Flights_2020.csv',
 '/home/Andree/Documents/Maestria/DatosMasivos/DatosMasivos/Datos/Combined_Flights_2019.csv',
 '/home/Andree/Documents/Maestria/DatosMasivos/DatosMasivos/Datos/Combined_Flights_2018.csv']

In [10]:
df = spark.read.option("header","true").csv(archivos_datos)

In [11]:
df = df.select(columnas)

In [12]:
df = df.persist(StorageLevel.DISK_ONLY)

24/10/21 00:07:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [13]:
df = df.withColumn("CRSDepTime", df["CRSDepTime"].cast(pysql.types.FloatType()))
df = df.withColumn("DepTime", df["DepTime"].cast(pysql.types.FloatType()))
df = df.withColumn("DepDelay", df["DepDelay"].cast(pysql.types.FloatType()))
df = df.withColumn("ArrTime", df["ArrTime"].cast(pysql.types.FloatType()))
df = df.withColumn("ArrDelay", df["ArrDelay"].cast(pysql.types.FloatType()))
df = df.withColumn("ActualElapsedTime", df["ActualElapsedTime"].cast(pysql.types.FloatType()))
df = df.withColumn("CRSElapsedTime", df["CRSElapsedTime"].cast(pysql.types.FloatType()))
df = df.withColumn("Distance", df["Distance"].cast(pysql.types.FloatType()))
df = df.withColumn("Year", df["Year"].cast(pysql.types.IntegerType()))
df = df.withColumn("Month", df["Month"].cast(pysql.types.IntegerType()))
df = df.withColumn("DayofMonth", df["DayofMonth"].cast(pysql.types.IntegerType()))
df = df.withColumn("DayOfWeek", df["DayOfWeek"].cast(pysql.types.IntegerType()))

df = df.withColumn("Cancelled", pysql.functions.when(df["Cancelled"] == "False",0).otherwise(1))
df = df.withColumn("Diverted", pysql.functions.when(df["Diverted"] == "False",0).otherwise(1))

In [14]:
df.show(truncate=False)

24/10/21 00:07:16 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+-----------------------------------------+----------+-------+--------+----------+-------+--------+-----------------+--------------+--------+----+-----+----------+---------+-----------+-------------------------------+------+---------------+------------------+------------------------+---------------+----+-------------+----------------+-------------------+-------------+---------+--------+
|Airline                                  |CRSDepTime|DepTime|DepDelay|CRSArrTime|ArrTime|ArrDelay|ActualElapsedTime|CRSElapsedTime|Distance|Year|Month|DayofMonth|DayOfWeek|Tail_Number|Flight_Number_Operating_Airline|Origin|OriginAirportID|OriginAirportSeqID|OriginCityName          |OriginStateName|Dest|DestAirportID|DestAirportSeqID|DestCityName       |DestStateName|Cancelled|Diverted|
+-----------------------------------------+----------+-------+--------+----------+-------+--------+-----------------+--------------+--------+----+-----+----------+---------+-----------+-------------------------------+---

In [15]:
df.printSchema()

root
 |-- Airline: string (nullable = true)
 |-- CRSDepTime: float (nullable = true)
 |-- DepTime: float (nullable = true)
 |-- DepDelay: float (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- ArrTime: float (nullable = true)
 |-- ArrDelay: float (nullable = true)
 |-- ActualElapsedTime: float (nullable = true)
 |-- CRSElapsedTime: float (nullable = true)
 |-- Distance: float (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Tail_Number: string (nullable = true)
 |-- Flight_Number_Operating_Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginAirportID: string (nullable = true)
 |-- OriginAirportSeqID: string (nullable = true)
 |-- OriginCityName: string (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestAirportID: string (nullable = true)
 |-- DestAir

In [31]:
df.count()

29193782

In [16]:
df.groupBy("Year").agg({
    "Cancelled" : "avg"
}).orderBy(pysql.functions.desc("avg(Cancelled)")).show(truncate=False)

+----+--------------------+
|Year|avg(Cancelled)      |
+----+--------------------+
|2020|0.059942493594194166|
|2022|0.030206570453799826|
|2019|0.0189860355396973  |
|2021|0.017588762507979014|
|2018|0.015532615099502383|
+----+--------------------+



In [17]:
df.groupBy("Year").agg({
    "Diverted" : "avg"
}).orderBy(pysql.functions.desc("avg(Diverted)")).show(truncate=False)

+----+---------------------+
|Year|avg(Diverted)        |
+----+---------------------+
|2019|0.0025694280696082544|
|2022|0.0025034830535529597|
|2018|0.0024527586900247333|
|2021|0.0023736226548356264|
|2020|0.0016746983561833125|
+----+---------------------+



In [18]:
spark.stop()